In this section we are addressing with optuna library in python. Optuna is a tool for optimizing the model in which this optimization is applying on validation accuracy. We try to explain this practical feature by examples in ML modeling.

In [1]:
import optuna

Example 1: Here we have a function like $f(x) = (x-3)^2$ that we're gonna optimize this function. Exact value for this optimization is three. Let's see how optuna is working.


In [ ]:
def example_1(trial):
    x = trial.suggest_float('x',-10,10) # here we suggest a float range for x
    y = (x-3)**2 # this is the function that we want to optimize it
    return y

study = optuna.create_study()
study.optimize(func = example_1, n_trials = 100, gc_after_trial = True )
study.best_params  # E. g {'x': 2.9841660899868985}

Example 2: This example deal with a function which has two parameter like $ f(w,b) = w^2 + (b+2)^2$ that we're gonna optimize it and lets see how $w$ and $b$ will optimize.

In [ ]:
def example_2(trial):
    w = trial.suggest_float('w', -10, 10)
    b = trial.suggest_float('b', -10, 10)
    y =  (w**2+(b+2)**2)
    return y

study = optuna.create_study()
study.optimize(func = example_2, n_trials= 200, gc_after_trial = True)
study.best_params # E. g {'w': 0.026869541290451715, 'b': -1.9468943029007124}

Example 3) In this example we want to import `iris` dataset and then we try to make a classification model base on that by using `Scikit-Learn`. By using optuna we have plan to optimize hyper parameters.

In [ ]:
import pandas as pd
import numpy as np
import sklearn.svm
import sklearn.datasets
import sklearn.model_selection 
from sklearn.metrics import accuracy_score

iris = sklearn.datasets.load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size = 0.8, random_state = 42)  # make train and test part for final evaluation

def example_3(trial):
    
    params = {
                'kernel' : trial.suggest_categorical('kernel', ['rbf', 'poly'])
            }

    clf = sklearn.svm.SVC(**params)
    score  = sklearn.model_selection.cross_val_score(estimator = clf, X = X_train, y = y_train, cv=5) # super-nice function for applying cross validation on model
    accuracy = score.mean()

    return accuracy

study = optuna.create_study()
study.optimize(func=example_3, n_trials= 100)
best_cls_params = study.best_params # E. g {'kernel': 'rbf'}

In [14]:
clf = sklearn.svm.SVC(**best_cls_params)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_pred = y_pred, y_true = y_test)
print(f'Accuracy of this model is {acc}')

Accuracy of this model is 1.0


Example 4: In this example, Tensorflow and MNIST are used to optimize the validation accuracy of hand-written digit recognition.Here we are optimizing the neural network architecture as well as the optimizer configuration.


In [17]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

BATCH_SIZE = 128

In [46]:
def get_mnist():

    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    x_train = x_train.astype('float32')/255
    x_valid = x_valid.astype('float32')/255

    y_train = y_train.astype('int32')
    y_valid = y_valid.astype('int32')

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(60000) # we are shuffling entire dataset
    train_ds = train_ds.batch(BATCH_SIZE) # Dataset is batched in smaller batches for improve the efficiency

    valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
    valid_ds = valid_ds.shuffle(10000)
    valid_ds = valid_ds.batch(BATCH_SIZE)

    return train_ds, valid_ds
